This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
# Cross Validation Classification Accuracy
from pandas import read_csv
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
# from sklearn.neighbors import KNeighborsClassifier
#import faiss
from numpy import isnan
#import matplotlib.pyplot as plot
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
import shutil
import os
from os import path
import ipynb.fs.full.our_functions_library as flib
from our_evaluate_sepsis_score import evaluate_performance

In [ ]:
# Global initialization, get all the data from file and generate all the needed variable
filename = 'raw_data/raw_data_400.csv' # use raw dataset
#filename = 'data.csv' # use raw dataset
originalData = read_csv(filename) # read csv data into DataFrame var raw
print("Data size:",originalData.shape)
Original_Uniq_ID= np.unique(originalData['Patient_id']) 
Uniq_ID = Original_Uniq_ID.copy()
print('Patient id size:',len(Uniq_ID))
X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']
# Initialize the empty array X_train, X_test, y_train, y_test
X_train = pd.DataFrame(columns = X_columns)
X_test = pd.DataFrame(columns = X_columns)
y_train = pd.DataFrame(columns = y_columns)
y_test = pd.DataFrame(columns = y_columns)
# Below are the lists for KNN results

KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []

KNN_F1Score_mean = []
KNN_F1Score_std = []

KNN_auroc_mean = []
KNN_auprc_mean = []

KNN_accuracy_mean = []
KNN_accuracy_std = []

KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]


fillmethod =""  

In [ ]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(Original_Uniq_ID,Uniq_ID) ):
    print("They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs    
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Original_Uniq_ID[0:25])
print("Uniq id:************************************")
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...\n",Uniq_ID[0:25])

In [ ]:
#fill the missing data with one function
print(filename)
print(len(Original_Uniq_ID))
#print(originalData)
if (originalData.isnull().values.any()):
    print('There is data missing in the original data set')
dataByPatient = originalData.groupby('Patient_id') # data grouped by Patient_id

In [ ]:
def Average(lst): 
    return sum(lst) / len(lst) 

#this function will clear everything in the trainng and test dataset
def clearAllDatasets():
    global X_train, X_test, y_train, y_test
    if not X_train.empty: 
        X_train = X_train[0:0]
    if not X_test.empty:
        X_test = X_test[0:0]
    if not y_train.empty:
        y_train = y_train[0:0]
    if not y_test.empty:
        y_test = y_test[0:0]
    
def printDataset():
    print('X train shape:',X_train.shape)
    print(X_train)
    print( 'Y train shape:',y_train.shape)
    print(y_train)
    print('X test shape:',X_test.shape)
    print(X_test)
    print( 'Y test shape:',y_test.shape)
    print(y_test)
    
def printDatasetType():
    print('X_train type:',type(X_train),'y_train type:',type(y_train) )
    print('X test shape:',type(X_test),'Y test shape:',type(y_test) )
    print('X_train data type:',(X_train.dtypes),'y_train data type:',(y_train.dtypes) )
    print('X_test data type:',(X_test.dtypes),'Y_test data type:',(y_test.dtypes) )
    
def KNN_reset():
    #KNN with different K size, 
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
def generateTrainDataSet(patienIds):
    global X_train, y_train
    for i in patienIds:
#     print('Patient_id',i,':\n',dataByPatient.get_group(i),'\n')
        X_train = X_train.append(dataByPatient.get_group(i).loc[:, X_columns])
        y_train = y_train.append(dataByPatient.get_group(i).loc[:, y_columns])
    return X_train, y_train
 
def generateTestDataSet(patienIds):
    global X_test, y_test
    for i in patienIds:
#     print('Patient_id',i,':\n',dataByPatient.get_group(i),'\n')
        X_test = X_test.append(dataByPatient.get_group(i).loc[:, X_columns])
        y_test = y_test.append(dataByPatient.get_group(i).loc[:, y_columns]) 
    return X_test, y_test

#Functions below are filling the missing data
#This function perform the KNN missing data filling for 
# import numba as nb
# @nb.jit
def KNNfilling(trainData,testData,K= 5, fillmethod=""):
    imputer = KNNImputer(n_neighbors = K)
    #imputer = FaissKNeighbors(k=K)
    # fit 
    imputer.fit(trainData)
    #transfer
#     x_train_impute=imputer.transform(trainData).round(3)
#     x_test_impute=imputer.transform(testData).round(3)
    x_train_impute=imputer.transform(trainData)
    x_test_impute=imputer.transform(testData)
    fillmethod= "KnnFill"
    return x_train_impute, x_test_impute, fillmethod   #This may cause error when the data is very large in size

def CalculateSOFAScore(X_train_impute, X_test_impute):
    pass


In [ ]:
def WriteToFiles(X_test_impute,y_test, NFold, fillmethod):
    x= pd.DataFrame(data=X_test_impute, columns=X_columns)
    y= pd.DataFrame(data=y_test, columns=y_columns)
    # tot= x.join(y)
    x.merge(y, how='inner', on='Patient_id')
    patient_size=np.unique(x['Patient_id'])
#     print(patient_size)
#     print(int(patient_size[2]))
    dirName=r'C:\Users\r0631\Documents\K-Fold\filled_data/'+str(fillmethod)+'/Fold'+str(NFold+1)
    if os.path.exists(dirName):
        shutil.rmtree(dirName)  # remove existing directory
    os.makedirs(dirName)
    for ind in range(len(patient_size)):
        #filename=r'output/Fold'+str(NFold+1)+'/p'+str(ind)+'.psv'
        id =int(patient_size[ind])
        filename=dirName+'/p'+str(id)+'.psv'
        patient = x.loc[x['Patient_id'] == id]
        patient.to_csv (filename, index = False, header=True, sep='|')
    print('Fold',NFold,'written away')
    
def CalcMean_Std (Data):
    print(Data)
    mean= np.mean(Data, axis=0)
    np.set_printoptions(precision=3, suppress=True)
    print('numpy mean\n',mean)
    Data_std = Data.std(axis = 0) 
    print('Data_mean\n',Data_std.round(3))
    
def plotKNNResultFigure(KValuestart,KvalueEnd,yValues,title,xlabel,ylabel):
    plt.figure(figsize=(10,6))
    plt.plot(range(KValuestart,KvalueEnd+1,5),yValues,color = 'blue',linestyle='dashed', 
             marker='o',markerfacecolor='red', markersize=10)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    print("Maximum ",ylabel,":-",max(yValues),"at K =", 1 + yValues.index(max(yValues)))
    

In [ ]:
# Code for K-fold algorithm, Before this step the missing data has been filled, 
def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes
    global X_train, X_test, y_train, y_test
    print("K Fold of ",KforKFold ," folds with KNN =",KforKNN)
    # initialisation of the array for storing the different intermediate results
    accuracy_model = []
    F1Score_model = []
    baseline_model= []
    auroc_model = []
    auprc_model = []
    physio_accuracy_model = []
    f_measure_model = []
    utility_score_model = []
    mean_train = 0
    positivepredictions = []
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
    #print("Patients number: ", len(Uniq_ID)) #print('idSets arrays',idSets)
    #This for loop is for Kfold, calculating the results for K times
    for i in range(KforKFold):
        start1 = time.time()
        clearAllDatasets() #first clear all the datasets
        print("for the",i+1,"th iteration",idSets[i])   
        X_test,y_test = generateTestDataSet(idSets[i])
        for j in range(KforKFold):
            if j != i:
                 X_train, y_train = generateTrainDataSet(idSets[j]) 
        #printDataset()
        #Now the train and test dataset is generated
        #we can begin to train the model wit the training set and evaaulate the performance with the test sett   
#         X_train = X_train.drop('time', 1) X_test = X_test.drop('time', 1)
        X_train=X_train.astype('float64')
        X_test=X_test.astype('float64')    
        YTest_copy = y_test  # variable of joining the filled data (X) and Y (Train_output) 
        patientID_ytest = y_test['Patient_id']
        y_train = y_train.drop('Patient_id', 1)
        y_test = y_test.drop('Patient_id', 1)      
        y_train=y_train.astype('float64')
        y_test=y_test.astype('float64')
        #print('YTest',YTest_copy.head())
        
        #fill the missing data
        if X_train.isnull().values.any() or X_test.isnull().values.any() :
            print("X_train or X_test contains NaN values, KNN/mean is performed.")
            #if there is missing value
            X_train_impute, X_test_impute, fillmethod = KNNfilling(X_train, X_test, KforKNN, fillmethod)
            #X_train_impute, X_test_impute,fillmethod = MeanFilling(X_train,X_test, fillmethod)            
        else:
            print("X_train or X_test have all been filled ")
            X_train_impute = X_train
            X_test_impute = X_test
            
        #Scale the data
#         scaler = preprocessing.StandardScaler()
#         scaler.fit(X_train_impute)
#         X_train_impute = scaler.transform(X_train_impute)
#         X_test_impute = scaler.transform(X_train_impute)
            
        #fit the model and predict
        model.fit(X_train_impute, y_train)
        y_predicted = model.predict(X_test_impute)   
        y_predicted_probobility = model.predict_proba(X_test_impute) 
        #transfer the outpu and evalute it
        y_labels = y_test.astype(int).to_numpy()
        y_predicted = y_predicted.astype(int)
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
        #print(y_labels)
        
#         # parameter for gridsearchcv
#         param_dict={
#             "criterion":['gini', 'entropy'],
#             "max_depth":range(1,10),
#             "min_samples_split":range(1,10),
#             "min_samples_leaf":range(1,5)
#         }
        
#         # finding best hyperparameter using gridsearchcv
#         grid=GridSearchCV(estimator=decision_tree,
#                           param_grid=param_dict,
#                           cv=10,
#                           verbose=1,
#                           n_jobs=-1)
#         grid.fit(X_train, y_train)
#         print("best_params:",grid.best_params_,"best_score",grid.best_score_,"\n")
       
        auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance(y_labels, y_predicted, y_predicted_probobility,patientID_ytest)
#         print("\nauroc",round(auroc,4),"auprc",round(auprc,4),"util_accuracy",round(accuracy1,4))
#         print("f_measure",round(f_measure,4),"utility_score",round(utility_score,4))  
        auroc_model.append(round(auroc,4))
        auprc_model.append(round(auprc,4))
        physio_accuracy_model.append(round(physio_accuracy,4))
        f_measure_model.append(round(f_measure,4))
        utility_score_model.append(round(utility_score,4))
        positivepredictions.append(np.sum(y_predicted))

#       num_rows= X_test_impute.shape[0]
#       scores = np.zeros(num_rows).astype('float64')# score parameter:output from get_sepsis_score
#       labels = np.zeros(num_rows).astype('float64')
        #WriteToFiles(X_train_impute,YTest_copy,i, fillmethod)
        #CalcMean_Std(X_train_impute)
        #take down the results
        accuracy_model.append((accuracy_score(y_test, y_predicted, normalize=True)*100).round(2))
        F1Score_model.append((f1_score(y_test, y_predicted)*100).round(2))
        baseline_model.append(round( (1 - float(y_test.mean()) )*100 , 2 ))
        #print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
        #baseline_model.append(((1-y_test.mean())*100).round(2))
        #print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
    #     print(accuracy_model)
    #     print(F1Score_model)    
        end1 = time.time()
        print("Time spent in this KFold iteration",round(end1- start1,2),"sec.\n")
        print("******************************************************************")
    print('Accuracy model:', accuracy_model)
    print('F1_score model:', F1Score_model)
    print('Baseline model:', baseline_model)
    
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print('Utility accuracy of model:', accuracy_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility score of model:', utility_score_model)
    
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_baseline_mean.append(np.mean(baseline_model))
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")


In [ ]:
def Print_DATA():
    print("KNN_UtilityScore_mean",KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std",KNN_UtilityScore_std)
    print("KNN_F1Score_mean",KNN_F1Score_mean)
    print("KNN_F1Score_std",KNN_F1Score_std)
    print("KNN_auroc_mean",KNN_auroc_mean)
    print("KNN_auprc_mean",KNN_auprc_mean)
    print("KNN_accuracy_mean",KNN_accuracy_mean)
    print("KNN_accuracy_std",KNN_accuracy_std)
    print("KNN_positiveprediction_mean",KNN_positiveprediction_mean)
    print("KNN_baseline_mean",KNN_baseline_mean)
    
    print(len(KNN_accuracy_mean))
    print(len(KNN_accuracy_std))
    print(len(KNN_F1Score_mean))
    print(len(KNN_F1Score_std))
    print(len(KNN_positiveprediction_mean))

def displayCurrentResult(KforKNNstart, KforKNNend):
    print((KNN_accuracy_mean))
    print((KNN_accuracy_std))
    print((KNN_F1Score_mean))
    print((KNN_F1Score_std))
    print((KNN_positiveprediction_mean))
    print(len(KNN_accuracy_mean))
    print(len(KNN_accuracy_std))
    print(len(KNN_F1Score_mean))
    print(len(KNN_F1Score_std))
    print(len(KNN_positiveprediction_mean))
    plotKNNResultFigure(KforKNN,KNN_accuracy_mean,"Mean Accuracy vs K",xlabel='K',ylabel="Mean Accuracy")
    plotKNNResultFigure(KforKNN,KNN_accuracy_std ,"Std of Accuracy vs K",xlabel='K',ylabel="Std of Accuracy")
    plotKNNResultFigure(KforKNN,KNN_F1Score_mean,"Mean F1 score vs K",xlabel='K',ylabel="Mean F1 score")
    plotKNNResultFigure(KforKNN,KNN_F1Score_std,"Std F1 score vs K",xlabel='K',ylabel="Std F1 score")
    plotKNNResultFigure(KforKNN,KNN_positiveprediction_mean,"Mean positive prediction vs K",xlabel='K',ylabel="Mean positive prediction") 
   
def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
    if(KforKNNend<=1):
        print("K must be a interger larger than 1")
        return
    KNN_reset()
    step=stepsize
    for i in range(KforKNNstart,KforKNNend+1,step) : 
        print("KNN of K = ",i)    
        KFold_patient(model,10,i)
    print("Now all the training is finished.")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_UtilityScore_mean,"Mean Utility Score vs K",xlabel='K',ylabel="mean Utility Score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_UtilityScore_std,"std Utility Score vs K",xlabel='K',ylabel="std Utility Score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_F1Score_mean,"Mean F1 score vs K",xlabel='K',ylabel="Mean F1 score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_F1Score_std,"Std F1 score vs K",xlabel='K',ylabel="Std F1 score")
    plotKNNResultFigure(KforKNNstart,KforKNNend,KNN_accuracy_mean,"Mean Accuracy vs K",xlabel='K',ylabel="Mean Accuracy")    

In [ ]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
dataByPatient = originalData.groupby('Patient_id') # data grouped by Patient_id

In [ ]:
Print_DATA()

In [ ]:
#Here is how to train the model
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2, class_weight="balanced")
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [ ]:
KFold_patient(decisionTreeModel,10,5)

In [ ]:
#random_State=2
# Accuracy model: [95.83, 92.21, 92.12, 95.05, 97.04, 81.35, 90.89, 97.42, 97.38, 96.77]
# F1_score model: [17.95, 4.41, 13.56, 2.47, 31.58, 5.39, 2.76, 28.07, 9.09, 8.16]
# Baseline model: [96.87, 99.04, 95.67, 97.06, 99.32, 98.41, 98.19, 98.17, 98.1, 97.92]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Utility accuracy of model: [95.83, 92.21, 92.12, 95.05, 97.04, 81.35, 90.89, 97.42, 97.38, 96.77]
# utility F1 of model: [0.1795, 0.0441, 0.1356, 0.0247, 0.3158, 0.0539, 0.0276, 0.2807, 0.0909, 0.0816]
# Utility score of model: [0.1297, -0.051, 0.1175, -0.002, 0.8935, 0.0092, -0.0333, 0.231, 0.0443, 0.046]


# random_stat=2'max_depth': 1,'min_samples_leaf': 1, 'min_samples_split': 2
# Accuracy model: [96.87, 99.04, 95.67, 97.06, 99.32, 98.41, 98.19, 98.17, 98.1, 97.92]
# F1_score model: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Baseline model: [96.87, 99.04, 95.67, 97.06, 99.32, 98.41, 98.19, 98.17, 98.1, 97.92]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0.7564, 0.9283, 0.5521, 0.6389, 0.4954, 0.7051, 0.6641, 0.8042, 0.6627, 0.6552]
# auprc of model: [0.2184, 0.0632, 0.0546, 0.0806, 0.0068, 0.0322, 0.0778, 0.1416, 0.1009, 0.3247]
# Utility accuracy of model: [96.87, 99.04, 95.67, 97.06, 99.32, 98.41, 98.19, 98.17, 98.1, 97.92]
# utility F1 of model: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Utility score of model: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# Accuracy model: [95.77, 85.13, 90.65, 93.98, 97.95, 73.43, 94.83, 93.83, 96.0, 98.06]
# F1_score model: [44.44, 11.43, 12.95, 23.81, 0.0, 7.41, 20.0, 27.94, 24.69, 42.55]
# Baseline model: [96.87, 99.04, 95.67, 97.06, 99.32, 98.41, 98.19, 98.17, 98.1, 97.92]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0.7564, 0.9249, 0.5505, 0.6389, 0.4931, 0.701, 0.6582, 0.7994, 0.6584, 0.6695]
# auprc of model: [0.2184, 0.0606, 0.0537, 0.0806, 0.0068, 0.0315, 0.0612, 0.1226, 0.0788, 0.2052]
# Utility accuracy of model: [95.77, 85.13, 90.65, 93.98, 97.95, 73.43, 94.83, 93.83, 96.0, 98.06]
# utility F1 of model: [0.4444, 0.1143, 0.1295, 0.2381, 0.0, 0.0741, 0.2, 0.2794, 0.2469, 0.4255]
# Utility score of model: [0.5365, 0.5735, 0.1281, 0.2819, -0.0581, 0.2135, 0.3188, 0.5782, 0.3271, 0.3317]


In [ ]:
Print_DATA()

In [ ]:
findBestKforKNN(decisionTreeModel,10,5,15)

In [ ]:
Print_DATA()

In [ ]:
findBestKforKNN(XGBoostModel,10,105,200)

In [ ]:
Print_DATA()

In [ ]:
findBestKforKNN(XGBoostModel,10,265,380,15)

In [ ]:
Print_DATA()